In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import html5lib
from bs4 import BeautifulSoup
import os

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

## Headless browser - without GUI Interface
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),chrome_options=chrome_options)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\User\.wdm\drivers\chromedriver\win32\99.0.4844.51]


In [2]:
url = "https://www.mpta.org.mm/prices_list.php?div_id=KXKe7L772TihagFZHaC9f39I1cxevaPeEnNlvFSUrbY%3D"
driver.get(url)
print(driver.title)

Prices List - Myanmar Petroleum Trade Association


In [7]:

def save_page(filename , html_file):
    with open(os.path.join('./html/', "{}.html".format(filename)), "w", encoding='utf-8') as f:
        f.write(html_file)



In [8]:
## find Price dropdown list 

## Select method didn't work in here because of the element inside <a>
# select = Select(driver.find_element_by_class_name('form-control form-control-sm'))

## working Select method inside detailed page
# select = Select(driver.find_element_by_name("dtList_length"))
# select.select_by_value("100")

## try above Select method with xpath to change 100 rows/page
driver.find_element_by_xpath("//select[@name='dtList_length']/option[text()='100']").click()

max_number = 133
page_number = 0

page_no = []
while page_number < max_number:
    try:
        page_list = driver.find_element(By.CLASS_NAME,"pagination")
        element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable(page_list.find_element(By.ID,"dtList_next")))
        print(element.get_attribute("class"))
        active_item = page_list.find_element(By.CLASS_NAME,"active")
        no = active_item.text
        if no:
            page_no.append(int(no))
        else:
            print("No Number!")
        html = driver.page_source
        save_page(no , html)
        element.click()
        print("After Click : " + str(page_number))
        time.sleep(5)
        page_number += 1
    except EC.NoSuchElementException:
        print("Something wrong!")
    # cls = driver.find_element(By.XPATH,xpath).click()
    # page_number += 1



C:\Users\User\AppData\Local\Temp\ipykernel_5920\2733488247.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//select[@name='dtList_length']/option[text()='100']").click()


paginate_button page-item next
After Click : 0
paginate_button page-item next
After Click : 1
paginate_button page-item next
After Click : 2
paginate_button page-item next
After Click : 3
paginate_button page-item next
After Click : 4
paginate_button page-item next
After Click : 5
paginate_button page-item next
After Click : 6
paginate_button page-item next
After Click : 7
paginate_button page-item next
After Click : 8
paginate_button page-item next
After Click : 9
paginate_button page-item next
After Click : 10
paginate_button page-item next
After Click : 11
paginate_button page-item next
After Click : 12
paginate_button page-item next
After Click : 13
paginate_button page-item next
After Click : 14
paginate_button page-item next
After Click : 15
paginate_button page-item next
After Click : 16
paginate_button page-item next
After Click : 17
paginate_button page-item next
After Click : 18
paginate_button page-item next
After Click : 19
paginate_button page-item next
After Click : 20
pa

In [13]:
## Read HTML file into Pandas Dataframes
## Append into SQLite DB 
## Ref - https://www.fullstackpython.com/blog/export-pandas-dataframes-sqlite-sqlalchemy.html

from sqlalchemy import create_engine
engine = create_engine('sqlite:///price.db', echo=True)
conn = engine.connect()
petroleum_table = "petrol"

from glob import glob
file_dir = glob('html/*.html')
for file in file_dir:
    name = os.path.basename(file).replace(".html", "_page")
    # print(name)
    try:
        df_table = pd.read_html(file, encoding='utf-8', attrs={'id':'dtList'})

        print(len(df_table))
        save_df = df_table[0]
        save_df.to_sql(petroleum_table, conn, if_exists='append')

    except:
        print("No table found on page {}".format(name))

1_page
1
2022-03-17 21:04:27,331 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("petrol")
2022-03-17 21:04:27,333 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-17 21:04:27,336 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("petrol")
2022-03-17 21:04:27,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-03-17 21:04:27,344 INFO sqlalchemy.engine.Engine 
CREATE TABLE petrol (
	"index" BIGINT, 
	"Srno." BIGINT, 
	"Effective Date" TEXT, 
	"Fuel Type" TEXT, 
	"Price Type" TEXT, 
	"Min" BIGINT, 
	"Max" BIGINT
)


2022-03-17 21:04:27,346 INFO sqlalchemy.engine.Engine [no key 0.00172s] ()
2022-03-17 21:04:27,374 INFO sqlalchemy.engine.Engine COMMIT
2022-03-17 21:04:27,376 INFO sqlalchemy.engine.Engine CREATE INDEX ix_petrol_index ON petrol ("index")
2022-03-17 21:04:27,377 INFO sqlalchemy.engine.Engine [no key 0.00116s] ()
2022-03-17 21:04:27,395 INFO sqlalchemy.engine.Engine COMMIT
2022-03-17 21:04:27,418 INFO sqlalchemy.engine.Engine INSERT INTO petrol ("index", "Srno.", "Ef